## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
sys.path.append('..') 
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Brae,GB,60.3964,-1.3530,50.99,84,99,16.49,light rain
1,1,Ponta Do Sol,PT,32.6667,-17.1000,70.00,73,60,3.47,broken clouds
2,2,Rikitea,PF,-23.1203,-134.9692,74.82,69,7,4.32,clear sky
3,3,Geraldton,AU,-28.7667,114.6000,50.41,53,75,16.11,broken clouds
4,4,Talnakh,RU,69.4865,88.3972,47.21,65,7,10.83,clear sky


In [3]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [4]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
16,16,Kapaa,US,22.0752,-159.3190,82.38,74,40,19.57,scattered clouds
19,19,Puerto Ayora,EC,-0.7393,-90.3518,76.96,87,88,5.99,overcast clouds
20,20,Cockburn Town,TC,21.4612,-71.1419,81.19,85,100,18.90,overcast clouds
21,21,Albany,US,42.6001,-73.9662,82.63,32,56,1.99,broken clouds
23,23,Georgetown,MY,5.4112,100.3354,77.90,88,40,0.00,moderate rain


In [5]:
# Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.count()

City                   192
Country                192
Max Temp               192
Current Description    192
Lat                    192
Lng                    192
Hotel Name             192
dtype: int64

In [8]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

print('----------Start Data Collection----------')
# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Could not find hotel for {row['City']}... skipping.")
        pass
print('----------Data Collection Complete----------')    

----------Start Data Collection----------
Could not find hotel for Albany... skipping.
Could not find hotel for Gaoua... skipping.
Could not find hotel for Algodones... skipping.
Could not find hotel for Merauke... skipping.
Could not find hotel for Sao Paulo De Olivenca... skipping.
Could not find hotel for Thinadhoo... skipping.
Could not find hotel for Aitape... skipping.
Could not find hotel for Ban Nahin... skipping.
Could not find hotel for Amapa... skipping.
Could not find hotel for Abu Samrah... skipping.
----------Data Collection Complete----------


In [13]:
# Drop the rows where there is no Hotel Name.
import numpy as np
clean_hotel_df = hotel_df.replace('', np.NaN, regex=True).dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
16,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Cockburn Town,TC,81.19,overcast clouds,21.4612,-71.1419,Osprey Beach Hotel
23,Georgetown,MY,77.90,moderate rain,5.4112,100.3354,Cititel Penang
25,Jaynagar-Majilpur,IN,81.84,scattered clouds,22.1758,88.4178,Joychanditala Sarbojonin Durga Utsav Committee


In [10]:
# Create the output File (CSV)
output_data_file = 'WeatherPy_Vacation.csv'
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))